In [11]:
import os 
import numpy as np 
import pickle

from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler



In [20]:
###### helper functions ####### 
def pkl_read(file_to_read, write_dir):
  os.chdir(write_dir)
  this = pickle.load(open(file_to_read, "rb"))
  # print(this)
  return this

def pkl_write(file_to_write, values_to_dump, write_dir):
  os.chdir(write_dir)
  with open(file_to_write, 'wb') as pickle_file:
      pickle.dump(values_to_dump, pickle_file)

def getdata_day(data,id,data_day):
  if id == 1:
    # var_name = data[0:data_day[id]+1,:,:]
    var_name = data[0:data_day[id],:,:]
  else:
    day_idx = np.where(np.array(list(data_day))== id)[0].astype(int) - 1
    #print(day_idx[0])
    prev_day_id = list(data_day)[day_idx[0]]
    #print(prev_day_id)
    var_name = data[data_day[prev_day_id]:data_day[id],:,:]
  return var_name

def gettarget_day(target_direction, id, day):
    if id == 1: 
      target = target_direction[:day[id]]
    else:
      day_idx = np.where(np.array(list(day))== id)[0].astype(int) - 1
      #print(day_idx[0])
      prev_day_id = list(day)[day_idx[0]]
      target = target_direction[day[prev_day_id]:day[id]] 
    return target


###### analysis functions #########

def calculate_PR(data, normalize_pr = True, normalize_data = 'zscore', sqrt_transform = False ):
    '''
    Args: 
    data (2D Numpy array): Neural data in format (n_timepoints, n_units)
    '''
    # n_t, n_u = data.shape
    if  normalize_data == 'zscore': 
        scaler_ = StandardScaler()
        standardized_data = scaler_.fit_transform(data)
        data = standardized_data
    if normalize_data  == 'demean':
        scaler_ = StandardScaler(with_mean = True, with_std = False)
        standardized_data = scaler_.fit_transform(data)
        data = standardized_data
#         print(scaler_.mean_)
        
    if sqrt_transform:
        data = np.sqrt(data + 0.375)  # see Kihlberg, 1972; 0.386 could also be a good value
            
    C_unit = np.cov(data.T)
#     print(C_unit.shape)
    eig_val, eig_vec = np.linalg.eig(C_unit)
    PR = (np.sum(eig_val))**2/ np.sum(eig_val**2)
    return (PR - 1) / (data.shape[1] - 1) if normalize_pr else PR

In [19]:
neural_recordings = pkl_read('neural_data.pkl', '.')
target_direction = pkl_read('target_labels.pkl', '.')
days_wtrials = pkl_read('trial_day_label.pkl', '.')

In [ ]:
# error bars for Global PR from 1000 draws on all trials
n_bs = 10 
normalize_data = 'demean'

g_normPR_all_bs = np.zeros((n_bs, len(days_wtrials)))
g_normPR_r_bs = np.zeros((n_bs, len(days_wtrials)))
g_normPR_nr_bs = np.zeros((n_bs, len(days_wtrials)))

l_normPR_all_bs = np.zeros((n_bs, len(days_wtrials), 8))
l_normPR_r_bs = np.zeros((n_bs, len(days_wtrials), 8))
l_normPR_nr_bs = np.zeros((n_bs, len(days_wtrials), 8))


for idraw in tqdm(range(n_bs)): 


      for id, iday in tqdm(enumerate(unique_bmi_day[days_wtrials])): 
            # per day
            data_all = getdata_day(neural_recordings,iday, datasize_day )
            target = gettarget_day(target_direction, iday, datasize_day)
            # print(np.unique(target))
            if id == early_day_idx:
                  data_early = data_all
            if id == late_day_idx:
                  data_late = data_all

            data = trial_concatenate_data(data_all)
            # print(data.shape)

            g_normPR_all_bs[idraw, id] = calculate_PR(data, normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)
            g_normPR_r_bs[idraw, id] = calculate_PR(data[:, :direct_idx_2.shape[0]], normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)
            g_normPR_nr_bs[idraw, id] = calculate_PR(data[:, direct_idx_2.shape[0]:], normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)

            # for it in np.unique(target).astype(int):
            #       it_idx = np.where(target[idx] == it)[0]
            #       data_it = data_idx[it_idx,:,:]
            #       data_it = trial_concatenate_data(data_it)
            #       # print(data.shape, it)

            #       l_normPR_all_bs[idraw, id, it-1] = calculate_PR(data_it, normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)
            #       l_normPR_r_bs[idraw, id, it-1] = calculate_PR(data_it[:, :direct_idx_2.shape[0]], normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)
            #       l_normPR_nr_bs[idraw, id, it-1] = calculate_PR(data_it[:, direct_idx_2.shape[0]:], normalize_pr= True, normalize_data=normalize_data, sqrt_transform=False)